<a href="https://colab.research.google.com/github/naufalfrdss/DeepLearning/blob/main/UAS/Chapter%2012/Custom_Models_and_Training_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dan Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

Model Subclassing: Wide and Deep

In [2]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output


Input, Kompilasi, dan Latih Model

In [3]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

model = WideAndDeepModel()
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

# Dummy data
X_train_A = np.random.rand(1000, 5)
X_train_B = np.random.rand(1000, 6)
y_train = np.random.rand(1000, 1)
y_aux = np.random.rand(1000, 1)

model.fit((X_train_A, X_train_B), (y_train, y_aux), epochs=5)


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.3012 - mse_loss: 0.2638
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1452 - mse_loss: 0.2115
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1301 - mse_loss: 0.1689
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1319 - mse_loss: 0.1480
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1183 - mse_loss: 0.1212


Custom Layer MyDenseLayer

In [4]:
class MyDenseLayer(keras.layers.Layer):
    def __init__(self, units=32, activation=None):
        super().__init__()
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros", trainable=True)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)


Gunakan Custom Layer dalam Model

In [5]:
custom_model = keras.models.Sequential([
    keras.layers.Input(shape=(8,)),
    MyDenseLayer(10, activation="relu"),
    MyDenseLayer(1)
])
custom_model.compile(optimizer="adam", loss="mse")

X_dummy = np.random.rand(100, 8)
y_dummy = np.random.rand(100, 1)

custom_model.fit(X_dummy, y_dummy, epochs=3)


Epoch 1/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3143 
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3389
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3267
